## GPT-4o Vision — AI-Generated Marketing Banners from Email Content
This notebook generates marketing banners using GPT-4o Vision (Image 1 API), based on just the **email content**. You can customize style, tone, and background with prompt tuning — no manual design required.


### Setup & Imports
Install and import the necessary libraries. Set up your `.env` file for the OpenAI API key and endpoint.


In [ ]:
import os
import json
import requests
import base64
from datetime import datetime

In [ ]:
OPENAI_API_KEY = os.getenv("AZURE_GPT_IMAGE_1_API_KEY")
OPENAI_ENDPOINT = os.getenv("AZURE_GPT_IMAGE_1_ENDPOINT")
OPENAI_DEPLOYMENT = os.getenv("AZURE_GPT_IMAGE_1_DEPLOYMENT_NAME")
OPENAI_API_VERSION = os.getenv("AZURE_GPT_IMAGE_1_API_VERSION", "2023-12-01-preview")

### Decode & Save Image Locally
This section decodes the base64 string and saves the image to disk as .png.

It also logs:

- Style, tone, background

- Prompt used

- Timestamp

- Shortened email body

In [ ]:
save_dir = "generated_banners"
os.makedirs(save_dir, exist_ok=True)

log_path = os.path.join(save_dir, "generation_log.json")
if not os.path.exists(log_path):
    with open(log_path, "w") as f:
        json.dump([], f)  # initialize log


In [ ]:

def save_banner_image_from_b64(image_b64, style, tone, background, email_body):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"banner_{style}_{tone}_{background}_{timestamp}.png"
    filepath = os.path.join(save_dir, filename)

    # Decode and save image from base64
    image_bytes = base64.b64decode(image_b64)
    with open(filepath, "wb") as handler:
        handler.write(image_bytes)

    # Log metadata
    with open(log_path, "r+") as log_file:
        logs = json.load(log_file)
        logs.append({
            "filename": filename,
            "style": style,
            "tone": tone,
            "background": background,
            "timestamp": timestamp,
            "email_body": email_body[:100],  # truncated
            
        })
        log_file.seek(0)
        json.dump(logs, log_file, indent=2)

    print(f"✅ Image saved: {filename}")


### Define Styles, Tones, and Backgrounds
Here we define prompt-tuning parameters that control the design language of the generated image.

In [10]:
# Styles to test
styles = ["realistic", "minimalist", "modern", "flat illustration", "3D render", "vibrant colors"]

# Tones to test
tones = ["professional", "fun", "luxurious", "bold", "elegant", "friendly"]

# Optional background concepts
backgrounds = ["abstract gradient", "product display", "seasonal theme", "digital pattern"]


### Prompt Builder
We define a function to build a structured image generation prompt using:
- Email body
- Channel (Email / WhatsApp / SMS)
- Style, tone, and background keywords


In [11]:
def build_prompt(email_body, channel, style, tone, background):
    return f"""Generate a high-quality marketing banner image.

Email Body: {email_body}
Channel: {channel}

Design Instructions:
- The banner should reflect a **{tone}** tone and follow a **{style}** visual style.
- Use a **{background}** background that matches the campaign theme.
- Ensure the design matches the channel:
    * Email: polished layout, strong visuals, branded colors
    * WhatsApp: bold elements, minimal text, high contrast
    * SMS: ultra-minimalist, single CTA focus

Avoid including too much text. Don’t generate screenshots or plain text banners — this is a visual marketing asset, not a text template."""


###  Image Generation with GPT-4o
Call the OpenAI API using the prompt. We use `b64_json` response format to get the image as base64, which we later decode and save.


In [24]:
def generate_banner_image(email_body, channel, style, tone, background):
    url = f"{OPENAI_ENDPOINT}/openai/deployments/{OPENAI_DEPLOYMENT}/images/generations?api-version={OPENAI_API_VERSION}"
    headers = {
            "Content-Type": "application/json",
            "api-key": OPENAI_API_KEY
        }
    

    payload = {
        "prompt": build_prompt(email_body, channel, style, tone, background),
        "size": "1024x1024",
        "quality": "high",  # if your model supports it
        "n": 1
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    return response


In [25]:
email_body = """Step into the new season with our limited-time Autumn Collection.

From cozy knits to stylish boots, discover pieces that combine comfort and elegance. Enjoy 30% off this week only.

Redefine your style today."""

### Loop Through Variants
Here, we loop over multiple combinations of styles, tones, and backgrounds using a nested loop to generate multiple banners for the same email body.

Each banner is saved with a timestamped filename.

In [ ]:
for style in styles:
    for tone in tones:
        for bg in backgrounds:
            response = generate_banner_image(
                email_body="Check out our latest product launch!",
                channel="email",
                style=style,
                tone=tone,
                background=bg
            )

            response_json = response.json()

            image_b64 = response_json['data'][0]['b64_json']


            save_banner_image_from_b64(image_b64, style, tone, bg, email_body)
            print(f"✅ Saved: {style} | {tone} | {bg}")